In [ ]:
# ============================================
# CELL 1: INSTALLATION (Run this first!)
# ============================================
!pip install -q groq chromadb pypdf2 openpyxl pandas gradio sentence-transformers

print("✅ All packages installed successfully!")
print("Now run the next cell with the application code...")

# ============================================
# CELL 2: APPLICATION CODE (Run after installation)
# ============================================

import os
import gradio as gr
from groq import Groq
import chromadb
from chromadb.utils import embedding_functions
import PyPDF2
import pandas as pd
from typing import List

class RAGSystem:
    def __init__(self):
        self.collection = None
        self.groq_client = None
        self.api_key = None
        self.doc_counter = 0

        # Initialize ChromaDB client
        self.chroma_client = chromadb.Client()

        # Initialize embedding function
        self.embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
            model_name="all-MiniLM-L6-v2"
        )

    def verify_api_key(self, api_key: str) -> tuple:
        """Verify if the Groq API key is valid"""
        try:
            client = Groq(api_key=api_key)
            # Test with a simple completion
            response = client.chat.completions.create(
                messages=[{"role": "user", "content": "Hi"}],
                model="llama-3.3-70b-versatile",
                max_tokens=10
            )
            self.groq_client = client
            self.api_key = api_key
            return "✅ API Key verified successfully!", True
        except Exception as e:
            return f"❌ API Key verification failed: {str(e)}", False

    def split_text(self, text: str, chunk_size: int = 1000, overlap: int = 200) -> List[str]:
        """Simple text splitter"""
        chunks = []
        start = 0
        text_length = len(text)

        while start < text_length:
            end = start + chunk_size
            chunk = text[start:end]
            chunks.append(chunk)
            start = end - overlap

        return chunks

    def extract_text_from_pdf(self, file_path: str) -> str:
        """Extract text from PDF file"""
        text = ""
        try:
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
        except Exception as e:
            print(f"Error reading PDF: {e}")
        return text

    def extract_text_from_excel(self, file_path: str) -> str:
        """Extract text from Excel file"""
        try:
            df = pd.read_excel(file_path)
            return df.to_string()
        except Exception as e:
            print(f"Error reading Excel: {e}")
            return ""

    def extract_text_from_txt(self, file_path: str) -> str:
        """Extract text from text file"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read()
        except Exception as e:
            print(f"Error reading text file: {e}")
            return ""

    def process_documents(self, files) -> str:
        """Process uploaded documents and create vector store"""
        if not files:
            return "❌ No files uploaded!"

        all_texts = []
        file_names = []

        try:
            for file in files:
                file_path = file.name
                file_ext = os.path.splitext(file_path)[1].lower()
                file_name = os.path.basename(file_path)

                if file_ext == '.pdf':
                    text = self.extract_text_from_pdf(file_path)
                elif file_ext in ['.xlsx', '.xls']:
                    text = self.extract_text_from_excel(file_path)
                elif file_ext == '.txt':
                    text = self.extract_text_from_txt(file_path)
                else:
                    continue

                if text.strip():
                    all_texts.append(text)
                    file_names.append(file_name)

            if not all_texts:
                return "❌ No valid text extracted from files!"

            # Combine all texts
            combined_text = "\n\n".join(all_texts)

            # Split text into chunks
            chunks = self.split_text(combined_text)

            # Create or recreate collection
            try:
                self.chroma_client.delete_collection("documents")
            except:
                pass

            self.collection = self.chroma_client.create_collection(
                name="documents",
                embedding_function=self.embedding_function
            )

            # Add documents to collection
            ids = [f"doc_{i}" for i in range(len(chunks))]
            self.collection.add(
                documents=chunks,
                ids=ids
            )

            self.doc_counter = len(chunks)

            return f"✅ Successfully processed {len(files)} file(s) with {len(chunks)} chunks!"

        except Exception as e:
            return f"❌ Error processing documents: {str(e)}"

    def query_documents(self, question: str, chat_history) -> tuple:
        """Query the documents using RAG"""
        if not self.groq_client:
            return "❌ Please verify your API key first!", chat_history

        if not self.collection or self.doc_counter == 0:
            return "❌ Please upload and process documents first!", chat_history

        if not question.strip():
            return "❌ Please enter a question!", chat_history

        try:
            # Retrieve relevant documents
            results = self.collection.query(
                query_texts=[question],
                n_results=min(3, self.doc_counter)
            )

            context = "\n\n".join(results['documents'][0])

            # Create prompt with context
            prompt = f"""Based on the following context, answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer:"""

            # Get response from Groq
            response = self.groq_client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": "You are a helpful assistant that answers questions based on the provided context. Always provide accurate answers from the context. If the answer is not in the context, say so."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                model="llama-3.3-70b-versatile",
                temperature=0.3,
                max_tokens=1024
            )

            answer = response.choices[0].message.content

            # Update chat history
            chat_history.append((question, answer))

            return "", chat_history

        except Exception as e:
            error_msg = f"❌ Error: {str(e)}"
            return error_msg, chat_history

# Initialize RAG system
rag_system = RAGSystem()

# Create Gradio interface
with gr.Blocks(title="RAG System with Groq LLM", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📚 RAG System with Groq LLM")
    gr.Markdown("Upload your documents (PDF, Excel, Text) and ask questions about them!")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 🔑 Step 1: API Key")
            api_key_input = gr.Textbox(
                label="Groq API Key",
                type="password",
                placeholder="Enter your Groq API key..."
            )
            verify_btn = gr.Button("Verify API Key", variant="primary")
            api_status = gr.Textbox(label="Status", interactive=False)

            gr.Markdown("### 📤 Step 2: Upload Documents")
            file_upload = gr.File(
                label="Upload Files (PDF, Excel, Text)",
                file_count="multiple",
                file_types=[".pdf", ".xlsx", ".xls", ".txt"]
            )
            process_btn = gr.Button("Process Documents", variant="primary")
            process_status = gr.Textbox(label="Processing Status", interactive=False)

        with gr.Column(scale=2):
            gr.Markdown("### 💬 Step 3: Ask Questions")
            chatbot = gr.Chatbot(label="Chat History", height=400)
            question_input = gr.Textbox(
                label="Your Question",
                placeholder="Ask a question about your documents...",
                lines=2
            )
            submit_btn = gr.Button("Submit Question", variant="primary")
            clear_btn = gr.Button("Clear Chat")

    gr.Markdown("---")
    gr.Markdown("💡 **Tips:** Make sure to verify your API key and process documents before asking questions!")

    # Event handlers
    verify_btn.click(
        fn=lambda key: rag_system.verify_api_key(key),
        inputs=[api_key_input],
        outputs=[api_status]
    )

    process_btn.click(
        fn=rag_system.process_documents,
        inputs=[file_upload],
        outputs=[process_status]
    )

    submit_btn.click(
        fn=rag_system.query_documents,
        inputs=[question_input, chatbot],
        outputs=[question_input, chatbot]
    )

    question_input.submit(
        fn=rag_system.query_documents,
        inputs=[question_input, chatbot],
        outputs=[question_input, chatbot]
    )

    clear_btn.click(
        fn=lambda: [],
        outputs=[chatbot]
    )

# Launch the app
print("🚀 Launching Gradio interface...")
demo.launch(debug=True, share=True)

✅ All packages installed successfully!
Now run the next cell with the application code...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-3185692952.py:220: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="RAG System with Groq LLM", theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-3185692952.py:246: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat History", height=400)
/tmp/ipython-input-3185692952.py:246: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(label="Chat History", height=400)


🚀 Launching Gradio interface...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2553bdf1d21b4b6dab.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
